# Breweries Per Capita
State: **Washington**

**instructions:**
Use the 2020 Census data to determine the number of breweries per capita by county for a state of your choice. 
The Census data are provided in their raw format, you will need to join the population data to the US_counties feature class. 
You will create a new feature class of counties with the per capita values as the final output in the Notebook. 

### - Import packages, define wksp, and define datasets

In [122]:
import arcpy
wksp = arcpy.env.workspace = r'C:\\Users\\dillongeiger\\Desktop\\Week8_Juniper\\GIS4085_Week8_Data\\GIS4085_Week8_Data.gdb'

census_csv = "DECENNIALPL2020.P1_data_with_overlays_2021-12-03T101336.csv"
counties = "US_counties"
states = "US_states"
breweries = "Open_Brewery"

print(wksp, census_cvs, counties, states, breweries)


C:\\Users\\dillongeiger\\Desktop\\Week8_Juniper\\GIS4085_Week8_Data\\GIS4085_Week8_Data.gdb DECENNIALPL2020.P1_data_with_overlays_2021-12-03T101336.csv US_counties US_states Open_Brewery


### - Join tables by adding census_csv to counties
use common **SFFGEOID**/**GEO_ID** fields

In [123]:
table_join = arcpy.management.AddJoin(counties, "AFFGEOID", census_cvs, "GEO_ID", 'KEEP_COMMON')
print(table_join)

US_counties_Layer


### - Select state and clip both counties and breweries to the results 

In [124]:
state_name = "Washington"
wc = """name = '{}'""".format(state_name)
state_sel = arcpy.management.SelectLayerByAttribute(states, 'NEW_SELECTION', wc)
print(state_sel)

US_states_Layer1


In [125]:
counties_clip = arcpy.analysis.Clip(table_join, state_sel, "WA_counties")
print(counties_clip)
print("The data is now prepared to preform analysis.")

C:\\Users\\dillongeiger\\Desktop\\Week8_Juniper\\GIS4085_Week8_Data\\GIS4085_Week8_Data.gdb\WA_counties
The data is now prepared to preform analysis.


In [126]:
brew_clip = arcpy.analysis.Clip(breweries, state_sel, "WA_breweries")
print(brew_clip)

C:\\Users\\dillongeiger\\Desktop\\Week8_Juniper\\GIS4085_Week8_Data\\GIS4085_Week8_Data.gdb\WA_breweries


### - Conduct analysis of sum of points within polygons
creates a new fc: **WA_counties_brew**

In [127]:
new_fc = "WA_counties_brew"
total_brew = arcpy.analysis.SummarizeWithin(counties_clip, brew_clip, new_fc)
print(total_brew)

C:\\Users\\dillongeiger\\Desktop\\Week8_Juniper\\GIS4085_Week8_Data\\GIS4085_Week8_Data.gdb\WA_counties_brew


###  - Add new field to the new FC and calcuate with per capity 
**P1_001N** is the total population of the county  
**Point_Count** is the total breweries with the county 

new field: **PER_CAPITA**

In [128]:
field_name = "PER_CAPITA"
new_f = arcpy.management.AddField(new_fc, field_name, "FLOAT")
print(new_f)

WA_counties_brew


In [129]:
P1_001N = "DECENNIALPL2020_P1_data_with_overlays_2021_12_03T101336_csv_P1_0"
fields = [P1_001N, "Point_Count", field_name]
cursor = arcpy.da.UpdateCursor(new_fc, fields)
for row in cursor:
    row[2] = int(row[1])/ int(row[0])
    cursor.updateRow(row)

print("The new field is populated with per capita values.")

The new field is populated with per capita values.


### - The final step is to read the results and print an output

In [144]:
s_fields = ["US_counties_NAMELSAD", field_name]
search_c = arcpy.da.SearchCursor(new_fc, s_fields)
for row in search_c:
    print("There are {} breweries per 10000 people in {}, WA".format(row[1] *10000, row[0]))

There are 0.41253679228248075 breweries per 10000 people in Pierce County, WA
There are 1.0808891238411888 breweries per 10000 people in Skagit County, WA
There are 0.7125213596737012 breweries per 10000 people in Okanogan County, WA
There are 0.5191540185478516 breweries per 10000 people in Spokane County, WA
There are 0.653094393783249 breweries per 10000 people in Kitsap County, WA
There are 0.536447660124395 breweries per 10000 people in Clark County, WA
There are 1.393145794281736 breweries per 10000 people in Ferry County, WA
There are 0.3966365329688415 breweries per 10000 people in Grays Harbor County, WA
There are 0.2709292857616674 breweries per 10000 people in Cowlitz County, WA
There are 0.0 breweries per 10000 people in Douglas County, WA
There are 0.5727692405343987 breweries per 10000 people in King County, WA
There are 0.46052708057686687 breweries per 10000 people in Island County, WA
There are 0.20845058315899223 breweries per 10000 people in Whitman County, WA
There 